<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [92]:
!pip3 install folium
!pip3 install wget

"pip3" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.
"pip3" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


In [93]:
import folium
import wget
import pandas as pd

In [94]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [95]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [............................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [96]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [97]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [98]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [99]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

##############################################################################
launch_sites = folium.map.FeatureGroup()
for lat, lng, in zip(launch_sites_df.Lat, launch_sites_df.Long):
    launch_sites.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )


##############################################################################

latitudes = list(launch_sites_df.Lat)
longitudes = list(launch_sites_df.Long)
labels = list(launch_sites_df['Launch Site'])

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(site_map)
    
site_map.add_child(launch_sites)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [100]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [101]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [102]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

marker_color = []

for i,e in enumerate(spacex_df["class"]):
    if e == 1:
        #spacex_df["marker_color"][i] = 'green'
        marker_color.append('green')
    else:
        #spacex_df["marker_color"][i] = 'red'
        marker_color.append('red')

spacex_df["marker_color"] = marker_color  


In [103]:
# Function to assign color to launch outcome
#def assign_marker_color(launch_outcome):
#    if launch_outcome == 1:
#        return 'green'
#    else:
#        return 'red'
#    
#spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
#spacex_df.tail(10)

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [104]:
#spacex_df[["marker_color"]][spacex_df["Launch Site"]=='CCAFS SLC-40'].value_counts()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(location=[record[1], record[2]], icon=folium.Icon(color=record[4]))
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [105]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [106]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [107]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163

coastline_lat_list = [28.56392, 28.56392, 28.56392, 34.63778]
coastline_lon_list = [-80.56806, -80.56806, -80.56806, -120.62438]

launch_site_cords_df = spacex_df[["Launch Site","Lat","Long"]].groupby(["Launch Site"]).mean()
launch_site_cords_df = launch_site_cords_df.reset_index()
launch_site_cords_df["Coast_Lat"] = coastline_lat_list
launch_site_cords_df["Coast_Long"] = coastline_lon_list

distance_coastline_list = []

for site in launch_site_cords_df["Launch Site"]:
    
    launch_site_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Lat"]
    launch_site_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Long"]
    coastline_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Coast_Lat"]
    coastline_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Coast_Long"]
    
    distance_coastline_list.append(calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon))

launch_site_cords_df["Distance"] = distance_coastline_list
launch_site_cords_df

,Launch Site,Lat,Long,Coast_Lat,Coast_Long,Distance
0,CCAFS LC-40,28.562302,-80.577356,28.56392,-80.56806,0.925854
1,CCAFS SLC-40,28.563197,-80.576820,28.56392,-80.56806,0.859551
2,KSC LC-39A,28.573255,-80.646895,28.56392,-80.56806,7.770865
3,VAFB SLC-4E,34.632834,-120.610745,34.63778,-120.62438,1.363694


*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [108]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example

marker_cluster2 = MarkerCluster()
site_map.add_child(marker_cluster2)

for i, record in launch_site_cords_df.iterrows():
    
    distance_marker = folium.Marker(
        location=[record[3], record[4]],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(record[5]),
            )
        )
    marker_cluster2.add_child(distance_marker)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [109]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

for i, record in launch_site_cords_df.iterrows():
    
    lines = folium.PolyLine(locations = [ (record[1],record[2]), (record[3],record[4]) ], weight=1)
    site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [110]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

railway_lat_list = [28.5721, 28.5721, 28.57384, 34.63741]
railway_lon_list = [-80.58527, -80.58527, -80.6542, -120.62342]
launch_site_cords_df["railway_Lat"] = railway_lat_list
launch_site_cords_df["railway_Long"] = railway_lon_list

Highway_lat_list = [28.56362, 28.56362, 28.57346, 34.66849]
Highway_lon_list = [-80.57082, -80.57082, -80.65546, -120.45764]
launch_site_cords_df["highway_Lat"] = Highway_lat_list
launch_site_cords_df["highway_Long"] = Highway_lon_list

city_lat_list = [28.61533, 28.61533, 28.61533, 34.6469]
city_lon_list = [-80.81473, -80.81473, -80.81473, -120.45786]
launch_site_cords_df["city_Lat"] = city_lat_list
launch_site_cords_df["city_Long"] = city_lon_list

distance_railway_list = []
distance_highway_list = []
distance_city_list = []

for site in launch_site_cords_df["Launch Site"]:
    
    launch_site_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Lat"]
    launch_site_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"Long"]

    railway_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"railway_Lat"]
    railway_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"railway_Long"]
    distance_railway_list.append(calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon))

    highway_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"highway_Lat"]
    highway_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"highway_Long"]
    distance_highway_list.append(calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon))

    city_lat = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"city_Lat"]
    city_lon = launch_site_cords_df[launch_site_cords_df["Launch Site"] == site].reset_index().loc[0,"city_Long"]
    distance_city_list.append(calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon))

launch_site_cords_df["railway_distance"] = distance_railway_list
launch_site_cords_df["highway_distance"] = distance_highway_list
launch_site_cords_df["city_distance"] = distance_city_list

launch_site_cords_df


,Launch Site,Lat,Long,Coast_Lat,Coast_Long,Distance,railway_Lat,railway_Long,highway_Lat,highway_Long,city_Lat,city_Long,railway_distance,highway_distance,city_distance
0,CCAFS LC-40,28.562302,-80.577356,28.56392,-80.56806,0.925854,28.57210,-80.58527,28.56362,-80.57082,28.61533,-80.81473,1.336172,0.655177,23.922402
1,CCAFS SLC-40,28.563197,-80.576820,28.56392,-80.56806,0.859551,28.57210,-80.58527,28.56362,-80.57082,28.61533,-80.81473,1.289185,0.588038,23.948782
2,KSC LC-39A,28.573255,-80.646895,28.56392,-80.56806,7.770865,28.57384,-80.65420,28.57346,-80.65546,28.61533,-80.81473,0.716507,0.836936,17.046309
3,VAFB SLC-4E,34.632834,-120.610745,34.63778,-120.62438,1.363694,34.63741,-120.62342,34.66849,-120.45764,34.64690,-120.45786,1.266704,14.559931,14.078268


In [111]:
marker_cluster3 = MarkerCluster()
site_map.add_child(marker_cluster3)

for i, record in launch_site_cords_df.iterrows():
    
    distance_marker = folium.Marker(
        location=[record[6], record[7]],
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(record[12]), ) )
    
    marker_cluster3.add_child(distance_marker)

    distance_marker = folium.Marker(
        location=[record[8], record[9]],
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(record[13]), ) )
    
    marker_cluster3.add_child(distance_marker)

    distance_marker = folium.Marker(
        location=[record[10], record[11]],
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(record[14]), ) )
    
    marker_cluster3.add_child(distance_marker)

site_map

In [112]:
#selected_launch_site = "CCAFS LC-40"
#selected_launch_site = "CCAFS SLC-40"
#selected_launch_site = "KSC LC-39A"
selected_launch_site = "VAFB SLC-4E"

launch_site_cords1 = [launch_site_cords_df[launch_site_cords_df['Launch Site']=='CCAFS LC-40'].Lat, launch_site_cords_df[launch_site_cords_df['Launch Site']=='CCAFS LC-40'].Long]
launch_site_cords2 = [launch_site_cords_df[launch_site_cords_df['Launch Site']=='CCAFS SLC-40'].Lat, launch_site_cords_df[launch_site_cords_df['Launch Site']=='CCAFS SLC-40'].Long]
launch_site_cords3 = [launch_site_cords_df[launch_site_cords_df['Launch Site']=='KSC LC-39A'].Lat, launch_site_cords_df[launch_site_cords_df['Launch Site']=='KSC LC-39A'].Long]
launch_site_cords4 = [launch_site_cords_df[launch_site_cords_df['Launch Site']=='VAFB SLC-4E'].Lat, launch_site_cords_df[launch_site_cords_df['Launch Site']=='VAFB SLC-4E'].Long]

site_map2 = folium.Map(location=launch_site_cords1, zoom_start=10)

circle = folium.Circle(launch_site_cords1, radius=10, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
marker = folium.map.Marker(
    launch_site_cords1,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',
        )
    )

circle2 = folium.Circle(launch_site_cords2, radius=10, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40'))
marker2 = folium.map.Marker(
    launch_site_cords2,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',
        )
    )

circle3 = folium.Circle(launch_site_cords3, radius=10, color='#d35400', fill=True).add_child(folium.Popup('KSC LC-39A'))
marker3 = folium.map.Marker(
    launch_site_cords3,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A',
        )
    )

circle4 = folium.Circle(launch_site_cords4, radius=10, color='#d35400', fill=True).add_child(folium.Popup('VAFB SLC-4E'))
marker4 = folium.map.Marker(
    launch_site_cords4,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
        )
    )

if selected_launch_site =="CCAFS LC-40":
    site_map2.add_child(circle)
    site_map2.add_child(marker)
if selected_launch_site =="CCAFS SLC-40":
    site_map2.add_child(circle2)
    site_map2.add_child(marker2)
if selected_launch_site =="KSC LC-39A":
    site_map2.add_child(circle3)
    site_map2.add_child(marker3)
if selected_launch_site =="VAFB SLC-4E":
    site_map2.add_child(circle4)
    site_map2.add_child(marker4)

for i, record in launch_site_cords_df.iterrows():
    
    if record[0] == selected_launch_site:
        
        distance_marker_coast = folium.Marker(
            location=[record[3], record[4]],
            icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                html='<div style="font-size: 10; color:#000000;"><b>%s</b></div>' % "{:10.1f} KM".format(record[5]),
            )
        )
        site_map2.add_child(distance_marker_coast)
        
        distance_marker_railway = folium.Marker(
            location=[record[6], record[7]],
            icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                html='<div style="font-size: 10; color:#000000;"><b>%s</b></div>' % "{:10.1f} KM".format(record[12]),
            )
        )
        site_map2.add_child(distance_marker_railway)
        
        distance_marker_highway = folium.Marker(
            location=[record[8], record[9]],
            icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                html='<div style="font-size: 10; color:#000000;"><b>%s</b></div>' % "{:10.1f} KM".format(record[13]),
            )
        )
        site_map2.add_child(distance_marker_highway)
        
        distance_marker_city = folium.Marker(
            location=[record[10], record[11]],
            icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                html='<div style="font-size: 10; color:#000000;"><b>%s</b></div>' % "{:10.1f} KM".format(record[14]),
            )
        )
        site_map2.add_child(distance_marker_city)
        
        lines = folium.PolyLine(locations = [ (record[1],record[2]), (record[3],record[4]) ], weight=1)
        site_map2.add_child(lines)

        lines = folium.PolyLine(locations = [ (record[1],record[2]), (record[6],record[7]) ], weight=1)
        site_map2.add_child(lines)

        lines = folium.PolyLine(locations = [ (record[1],record[2]), (record[8],record[9]) ], weight=1)
        site_map2.add_child(lines)

        lines = folium.PolyLine(locations = [ (record[1],record[2]), (record[10],record[11]) ], weight=1)
        site_map2.add_child(lines)
    else: pass

site_map2

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
